## Introduction
This notebook presents a simple example of an ODE problem that I know can be solved as is. I would like to solve it in its 'inverse' form -- theoretically this is the same function, but I am struggling to get it working.

In [1]:
using JuMP
using Ipopt
using DataFrames, CSV
using Parameters
using DifferentialEquations
using Distributions
using NLsolve

### The following are "helper functions". They are unchanged between the two versions

In [2]:
## Helper Functions

function getCE(gamma, alpha, s, q_b, q_e , c, sigma_sq)
    b = getBidVec( gamma, alpha, s, q_b, q_e , c, sigma_sq)    

    b_min_c = b - (alpha*c);

    ce = (q_b' * b_min_c) - (gamma/2) * ((sigma_sq .* b_min_c)' * b_min_c)
end

function getBoundarySMax(alpha_max, gamma, q_b, q_e , c, sigma_sq)
    function ce_fun(svec)
        s_max_var = svec[1]
        return getCE(gamma, alpha_max, s_max_var, q_b, q_e , c, sigma_sq) 
    end
    starting_point = alpha_max*1.01*(c' * q_b)

    s_max_sol = nlsolve(ce_fun, [starting_point]; inplace = false)
    return s_max_sol.zero[1]
end



function getBidVec( gamma, alpha, s, q_b, q_e , c, sigma_sq)
    # Compute the optimal unit bid vector for each score by solving the quadratic optimization problem
    T = length(q_e)
    if sum(sigma_sq) > 0

        m = Model(Ipopt.Optimizer)
        set_optimizer_attribute(m, "print_level", 0)

        @variables m begin
        b[i=1:T] >= 0
        end

        @objective( m,
                    Max,
                    (sum((q_b[i] * (b[i] - alpha*c[i]) - (gamma / 2) * sigma_sq[i] * (b[i] - alpha*c[i])^2) for i=1:T ))
                 )

              @constraint(m, sum( b[i] * q_e[i] for i=1:T ) == s)

           optimize!(m)

           b_min = value.(b)
   else
       (maxitem,maxindex) = findmax(q_b./q_e)
       b_min = zeros(T)
       b_min[maxindex] = s/q_e[maxindex]
   end

    return b_min

end


function getDbDs(gamma, alpha, b, q_b, q_e , c, sigma_sq)
#     %Computes anayltical derivative of the optimal bid function of item t
#     % w.r.t. score s given alpha

    T = length(q_e)
    pos_b = [ifelse(b[t] > 0, 1, 0) for t=1:T]

    if (sum(sigma_sq) > 0)
        denom = 1.0/(sum((q_e[t]^2 * pos_b[t]) / sigma_sq[t] for t in 1:T))
        db_ds = [((q_e[t] * pos_b[t] / sigma_sq[t]) * denom) for t in 1:T]'
    else
        db_ds = [ifelse(b[t] > 0, (1.0/q_e[t]), 0) for t=1:T]'
    end

    return(db_ds)

end

function f_over_one_minus_F_uniform(z, a, b)
    if z <= a 
        out = 0
    elseif z >= b
        out = 0
    else 
        out =  1.0/(b - z) 
    end
end


f_over_one_minus_F_uniform (generic function with 1 method)

### Here is the "frontward" ODE (that works -- uncomment the print line to see du)

In [3]:
function ds_da_ode( s, alpha, gamma, c, q_e, q_a, sigma_sq ) ## By the way, I couldn't get an in-place formulation of this to work
                                                                ##  (see the end of the notebook -- not my main problem though)
    b = getBidVec( gamma, alpha, s, q_a, q_e , c, sigma_sq)
    db_ds = getDbDs( gamma, alpha, b, q_a, q_e , c, sigma_sq)

    b_min_c = b - (alpha*c);

    profit_ce = (q_a' * b_min_c) - (gamma)*0.5 * ((sigma_sq .* b_min_c)' * b_min_c)
    profit_term = exp(gamma*profit_ce) - 1;

    f_over_oneMinusF = f_over_one_minus_F_uniform(alpha, 0.5, 2.0)

    dce = db_ds * ((gamma .* q_a) - (gamma^2 .* sigma_sq .* b_min_c) );
    
    ds_da = f_over_oneMinusF * profit_term / dce; # Note: this should be a scalar
#     println("ds_da: ", ds_da)
    
    return(ds_da)

end


ds_da_ode (generic function with 1 method)

### Simple Numerical Example

In [4]:
qa = [12; 40];
qe = [10; 50];
c = [5;10];
sigmasq = [3, 5];
gamma = 1.5;

alpha_max = 2;
alpha_min = 0.5;

alpha_span = (alpha_max,alpha_min);

s_max_computed = getBoundarySMax(alpha_max, gamma, qa, qe , c, sigmasq)

ode_forward_subs(s, p, alpha) = ds_da_ode( s, alpha, gamma, c, qe, qa, sigmasq )
ode_forward_prob = ODEProblem(ode_forward_subs, s_max_computed, alpha_span)

sol_forward =  DifferentialEquations.solve(ode_forward_prob, Rosenbrock23(autodiff=false))




******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



retcode: Success
Interpolation: specialized 2nd order "free" stiffness-aware interpolation
t: 352-element Array{Float64,1}:
 2.0               
 1.9999955327240262
 1.9999940375182141
 1.9999848532038238
 1.999973530876456 
 1.9999289793374397
 1.9997807959586216
 1.9995998230587975
 1.9992061525765428
 1.9986377696629305
 1.997730330980752 
 1.9964993173161234
 1.9948686765448682
 ⋮                 
 0.5329076291496756
 0.5297482260480739
 0.5265888229464722
 0.5234294198448706
 0.5202700167432689
 0.5171106136416672
 0.5139512105400655
 0.5107918074384639
 0.5076324043368622
 0.5044730012352605
 0.5013135981336588
 0.5               
u: 352-element Array{Float64,1}:
 1097.9867070962464 
 1098.0887265142217 
 1098.0611955926092 
 1098.0113604203723 
 1097.9957693836875 
 1097.9730511793405 
 1097.9275853824045 
 1097.875173179889  
 1097.7592381018837 
 1097.5839434897146 
 1097.2846923035122 
 1096.842431628268  
 1096.201396032643  
    ⋮               
  296.6716943365059 
  294.93

## Now I'd like to solve the "inverse" of this ODE -- da_ds instead of ds_da 
### (I need this for a more complex version of the problem)

In [7]:
function inverse_ds_da_ode( s, alpha, gamma, c, q_e, q_a, sigma_sq )
    
    println("alpha: ", alpha, "; s: ", s)
    b = getBidVec( gamma, alpha, s, q_a, q_e , c, sigma_sq)
    db_ds = getDbDs( gamma, alpha, b, q_a, q_e , c, sigma_sq)

    b_min_c = b - (alpha*c);

    profit_ce = (q_a' * b_min_c) - (gamma)*0.5 * ((sigma_sq .* b_min_c)' * b_min_c)
    profit_term = exp(gamma*profit_ce) - 1;
    println("profit_term: ", profit_term)

    f_over_oneMinusF = f_over_one_minus_F_uniform(alpha, 0.5, 2.0)
    println("f_over_oneMinusF: ", f_over_oneMinusF)
    
    dce = db_ds * ((gamma .* q_a) - (gamma^2 .* sigma_sq .* b_min_c) );
    println("dce: ", dce)
    
    ds_da = f_over_oneMinusF * profit_term / dce; # Note: this should be a scalar
    println("ds_da: ", ds_da)
    
    da_ds = ds_da < 0 ? 0 : 1.0 / ds_da
    
    return(da_ds)

end

inverse_ds_da_ode (generic function with 1 method)

In [8]:
s_min_from_forward_solve = 278.5905933915064

alpha_init = alpha_min

s_span = (s_min_from_forward_solve, s_max_computed)


ode_backward_subs(alpha, p, s) = inverse_ds_da_ode( s, alpha, gamma, c, qe, qa, sigmasq )
ode_backward_prob = ODEProblem(ode_backward_subs, alpha_init, s_span)

sol_backward =  DifferentialEquations.solve(ode_backward_prob, Rosenbrock23(autodiff=false))



alpha: 0.5; s: 278.5905933915064
profit_term: 1007.444676029317
f_over_oneMinusF: 0
dce: 1.2223521841295824
ds_da: 0.0
alpha: 0.5; s: 278.5905933915064
profit_term: 1007.444676029317
f_over_oneMinusF: 0
dce: 1.2223521841295824
ds_da: 0.0
alpha: NaN; s: 278.5905933915064


ErrorException: Invalid coefficient NaN on variable b[1].

### Inspecting, it seems like maybe the problem is the 'f_over_oneMinusF' term at the initial condition, but shifting this a bit doesn't help [enough]

In [9]:
s_min_from_forward_solve = 278.5905933915064

alpha_init = alpha_min + 0.00001

s_span = (s_min_from_forward_solve, s_max_computed)


ode_backward_subs(alpha, p, s) = inverse_ds_da_ode( s, alpha, gamma, c, qe, qa, sigmasq )
ode_backward_prob = ODEProblem(ode_backward_subs, alpha_init, s_span)

sol_backward =  DifferentialEquations.solve(ode_backward_prob, Rosenbrock23(autodiff=false))



alpha: 0.50001; s: 278.5905933915064
profit_term: 1000.6876408713628
f_over_oneMinusF: 0.666671111140741
dce: 1.2223753872545824
ds_da: 545.7648676507391
alpha: 0.50001; s: 278.5905933915064
profit_term: 1000.6876408713628
f_over_oneMinusF: 0.666671111140741
dce: 1.2223753872545824
ds_da: 545.7648676507391
alpha: 0.5050100999999999; s: 281.31947230624684
profit_term: 975.0906157098308
f_over_oneMinusF: 0.6689008400658759
dce: 1.2224647238642707
ds_da: 533.5441745320052
alpha: 0.50001; s: 278.59059754282976
profit_term: 1000.6927239236486
f_over_oneMinusF: 0.666671111140741
dce: 1.2223753697411868
ds_da: 545.7676477151903
alpha: 0.50001; s: 278.5905933915064
profit_term: 1000.6876408713628
f_over_oneMinusF: 0.666671111140741
dce: 1.2223753872545824
ds_da: 545.7648676507391
alpha: 0.5000100149011611; s: 278.5905933915064
profit_term: 1000.6776058674764
f_over_oneMinusF: 0.6666711177635677
dce: 1.222375421829933
ds_da: 545.7593846462802
alpha: 0.50001; s: 278.5905933915064
profit_term: 10

ErrorException: Invalid coefficient -Inf on variable b[1].

### By the way, I couldn't figure out why this formulation wouldn't work: -- Really seems like it should...

In [7]:
function ds_da_ode!( ds, s, p, alpha ) ## a for alpha

    gamma, c, q_e, q_a, sigma_sq = p
    
    b = getBidVec( gamma, alpha, s, q_a, q_e , c, sigma_sq)
    db_ds = getDbDs( gamma, alpha, b, q_a, q_e , c, sigma_sq)

    b_min_c = b - (alpha*c);

    profit_ce = (q_a' * b_min_c) - (gamma)*0.5 * ((sigma_sq .* b_min_c)' * b_min_c)
    profit_term = exp(gamma*profit_ce) - 1;

    f_over_oneMinusF = f_over_one_minus_F_uniform(alpha, 0.5, 2.0)

    dce = db_ds * ((gamma .* q_a) - (gamma^2 .* sigma_sq .* b_min_c) );
    
    ds = f_over_oneMinusF * profit_term / dce; # Note: this should be a scalar
    println("ds_da: ", ds)
    

end


ode_forward_clean = ODEProblem(ds_da_ode!, s_max_computed, alpha_span, [gamma, c, qe, qa, sigmasq])
sol_forward_clean =  DifferentialEquations.solve(ode_forward_clean, Rosenbrock23(autodiff=false))


MethodError: MethodError: no method matching similar(::Float64)
Closest candidates are:
  similar(!Matched::ZMQ.Message, !Matched::Type{T}, !Matched::Tuple{Vararg{Int64,N}} where N) where T at /Users/svass/.julia/packages/ZMQ/KdNw3/src/message.jl:93
  similar(!Matched::BenchmarkTools.BenchmarkGroup) at /Users/svass/.julia/packages/BenchmarkTools/eCEpo/src/groups.jl:24
  similar(!Matched::DataFrames.StackedVector, !Matched::Type, !Matched::Union{Integer, AbstractUnitRange}...) at /Users/svass/.julia/packages/DataFrames/S3ZFo/src/abstractdataframe/reshape.jl:329
  ...